CS 585 – Fall 2023 – Homework 2 (100 total points) Due Wednesday September 27, 11:59pm

GOALS

• To perform a text classification experiment and examine the results

• To become familiar the scikit-learn machine learning library

• Optional exercise: To gain hands-on experience with word embeddings

DATA

For this homework you will build a text classification model to identify clickbait, or text headlines whose main purpose is to attract attention and entice readers to follow a link.

You will use data that has been created and shared by other NLP researchers on GitHub:

• Positive examples: https://github.com/pfrcks/clickbait-detection/blob/master/clickbait

• Negative examples: https://github.com/pfrcks/clickbait-detection/blob/master/not-clickbait

You do not need to read any research papers on clickbait detection to complete this homework, but you may enjoy reading the original publication that shared this dataset.

TOOLS

In this homework you are asked to use some functions from scikit-learn , an open-source python library widely used by data scientists. You should use Python to complete this homework assignment. Other programming languages will not be accepted.

NOTE ON MODEL PERFORMANCE

The goal of is homework is to execute an NLP experiment and present the results clearly to others (in this case, the class TAs) can read clearly. Different students will see different results, if you see lower metrics than others be assured that this will not lower your grade.

WHAT TO SUBMIT

Please upload or submit the following in Blackboard:

• For Problems 1 and 3-7, please upload to Blackboard: o One Jupyter notebook (.ipynb file) with cell output, showing your work for both datasets. o A PDF copy of the exact same notebook (same code and same output)

• For Problems 2 and 8 (no-code Q&A problems): Enter your written answers in Blackboard with your HW submission.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
import re

# Problem 1

• Using Python, read in the 2 clickbait datasets (See section DATA), and combine both into a single, shuffled dataset. (One function to shuffle data is numpy.random.shuffle)

• Next, split your dataset into train, test, and validation datasets. Use a split of 72% train, 8% validation, and 20% test. (Which is equivalent to a 20% test set, and the remainer split 90%/10% for train and validation). o If you prefer, you may save each split as an index (list of row numbers) rather than creating 3 separate datasets.

• What is the "target rate" of each of these three datasets?

In [2]:
import pandas as pd

# Define file paths
clickbait_file_path = '/Users/pradaapss/Downloads/clickbait.txt'
non_clickbait_file_path = '/Users/pradaapss/Downloads/not-clickbait.txt'

# Read the datasets into Pandas DataFrames
clickbait_df = pd.read_csv(clickbait_file_path, delimiter='\t', header=None)
non_clickbait_df = pd.read_csv(non_clickbait_file_path, delimiter='\t', error_bad_lines=False, header=None)

# Display the clickbait DataFrame
display(clickbait_df)

# Display the non-clickbait DataFrame
display(non_clickbait_df)

/var/folders/z4/hpxmr8tj43bbn9xbcynvfny80000gn/T/ipykernel_9930/2603116280.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  non_clickbait_df = pd.read_csv(non_clickbait_file_path, delimiter='\t', error_bad_lines=False, header=None)
Skipping line 700: expected 1 fields, saw 2



,0
0,"Man repairs fence to contain dog, hilarity ens..."
1,Long-Term Marijuana Use Has One Crazy Side Eff...
2,The water from his ear trickles into the bucke...
3,You'll Never Guess What Nick Jonas Does in the...
4,How Cruise Liners Fill All Their Unsold Cruise...
...,...
809,Mark your calendars: Jon Stewart announces the...
810,OITNB's Taylor Schilling and Carrie Brownstein...
811,Researchers have discovered the average penis ...
812,Why it may be smart to wait to put on sunscree...


,0
0,Congress Slips CISA Into a Budget Bill That's ...
1,DUI Arrest Sparks Controversy
2,It’s unconstitutional to ban the homeless from...
3,A Government Error Just Revealed Snowden Was t...
4,A toddler got meningitis. His anti-vac parents...
...,...
1568,US releases Guantánamo prisoner after 14 years...
1569,Loophole means ecstasy and loads of other drug...
1570,Astronomers Watch a Supernova and See Reruns
1571,"In Indian Rapists’ Neighborhood, Smoldering An..."


In [3]:
# Label the data as "text"
clickbait_df.columns = ["text"]
non_clickbait_df.columns = ["text"]


# Add a "label" column for clickbait (1) and non-clickbait (0)
clickbait_df["label"] = 'clickbait'
non_clickbait_df["label"] = 'non-clickbait'

# Combine both datasets into a single DataFrame
combined_df = pd.concat([clickbait_df, non_clickbait_df], ignore_index=True)

# Shuffle the combined dataset
np.random.shuffle(combined_df.values)

# Define the proportions for train, validation, and test splits
train_ratio = 0.72
validation_ratio = 0.08
test_ratio = 0.20

# Calculate the number of samples for each split
total_samples = len(combined_df)
train_samples = int(train_ratio * total_samples)
validation_samples = int(validation_ratio * total_samples)
test_samples = total_samples - train_samples - validation_samples

# Create index lists for each split
train_indices = list(range(train_samples))
validation_indices = list(range(train_samples, train_samples + validation_samples))
test_indices = list(range(train_samples + validation_samples, total_samples))

display(clickbait_df)

,text,label
0,"Man repairs fence to contain dog, hilarity ens...",clickbait
1,Long-Term Marijuana Use Has One Crazy Side Eff...,clickbait
2,The water from his ear trickles into the bucke...,clickbait
3,You'll Never Guess What Nick Jonas Does in the...,clickbait
4,How Cruise Liners Fill All Their Unsold Cruise...,clickbait
...,...,...
809,Mark your calendars: Jon Stewart announces the...,clickbait
810,OITNB's Taylor Schilling and Carrie Brownstein...,clickbait
811,Researchers have discovered the average penis ...,clickbait
812,Why it may be smart to wait to put on sunscree...,clickbait


In [4]:
def calculate_and_print_target_rate(df, name):
    clickbait_count = (df['label'] == 'clickbait').sum()
    total_count = len(df)
    target_rate = (clickbait_count / total_count) * 100
    print(f"{name} Target Rate: {target_rate:.4f}%")

# Split the combined dataset into train, validation, and test DataFrames
train_df = combined_df.iloc[train_indices]
validation_df = combined_df.iloc[validation_indices]
test_df = combined_df.iloc[test_indices]

# Calculate and print the target rates
calculate_and_print_target_rate(train_df, "Train")
calculate_and_print_target_rate(validation_df, "Validation")
calculate_and_print_target_rate(test_df, "Test")


Train Target Rate: 35.6228%
Validation Target Rate: 31.0526%
Test Target Rate: 29.8539%


# Problem 2

Baseline Performance (10 pts – Answer in Blackboard)

• Assume you have a trivial baseline classifier that flags every text presented to it as clickbait.
What is the precision, recall, and F1-score of such a classifier on your test set? Do you think
there is another good baseline classifier that would give you higher F-1 score? 

# Problem 3

Training a single Bag-of-Words (BOW) Text Classifier (20 pts)

• Using scikit-learn pipelines module, create a Pipeline to train a BOW naïve bayes model. We
suggest the classes CountVectorizer and MultinomialNB. Include both unigrams and bigrams in
your model in your vectorizer vocabulary (see parameter: ngram_range)

• Fit your classifier on your training set

• Compute the precision, recall, and F1-score on both your training and validation datasets using
functions in sklearn.metrics. Show results in your notebook. Use "clickbait" is your target class
(I.e., y=1 for clickbait and y=0 for non-clickbait)
ALTERNATIVE: If you are already well-versed in Naïve Bayes, you may select another bag-of-words
classifier for this problem. Your method should have some way to select top features or key indicators,
mapped to words or n-grams in your vocabulary, so that you can complete the remaining problems 

In [5]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

def evaluate_model(train_df, validation_df, pos_label='clickbait'):
    # Create a pipeline with CountVectorizer and MultinomialNB
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(ngram_range=(1, 2))),
        ('classifier', MultinomialNB())
    ])
    
    # Split the data into training and validation sets
    X_train, y_train = train_df['text'], train_df['label']
    X_validation, y_validation = validation_df['text'], validation_df['label']
    
    # Fit the classifier on the training set
    pipeline.fit(X_train, y_train)

    # Predict on training and validation sets
    y_train_pred = pipeline.predict(X_train)
    y_validation_pred = pipeline.predict(X_validation)

    # Compute precision, recall, and F1-score for training and validation sets
    metrics_train = precision_recall_fscore_support(y_train, y_train_pred, pos_label=pos_label, average='binary')
    metrics_validation = precision_recall_fscore_support(y_validation, y_validation_pred, pos_label=pos_label, average='binary')

    # Print the results
    print("Training Precision: {:.4f}%".format(metrics_train[0] * 100))
    print("Training Recall: {:.4f}%".format(metrics_train[1] * 100))
    print("Training F1-Score: {:.4f}%".format(metrics_train[2] * 100))
    print('\t')
    print("Validation Precision: {:.4f}%".format(metrics_validation[0] * 100))
    print("Validation Recall: {:.4f}%".format(metrics_validation[1] * 100))
    print("Validation F1-Score: {:.4f}%".format(metrics_validation[2] * 100))

# Assuming you have train_df and validation_df
evaluate_model(train_df, validation_df, pos_label='clickbait')


Training Precision: 99.0291%
Training Recall: 100.0000%
Training F1-Score: 99.5122%
	
Validation Precision: 83.3333%
Validation Recall: 93.2203%
Validation F1-Score: 88.0000%


# Problem 4

Hyperparameter Tuning (20 pts)
Using the ParameterGrid class, run a small grid search where you vary at least 3 parameters of your
model

• max_df for your count vectorizer (threshold to filter document frequency)

• alpha or smoothing of your NaïveBayes model

• One other parameter of your choice. This can be non-numeric; for example, you can consider a
model with and without bigrams (see parameter "ngram" in class CountVectorizer)

Show metrics on your validation set for precision, recall, and F1-score. If your grid search is very large
(>50 rows) you may limit output to the highest and lowest results. 
ALTERNATIVE – If you used a method other than Naïve Bayes in Problem 3, then be sure it is clear what
metrics you tuned in Problem 4.

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline

# Define functions for preprocessing and evaluation
def preprocess_data(df):
    X = df['text']
    y = df['label']
    return X, y

def evaluate_pipeline(pipeline, X, y):
    y_pred = pipeline.predict(X)
    class_labels = ['clickbait', 'non-clickbait']
    precision = precision_score(y, y_pred, labels=class_labels, average=None)
    recall = recall_score(y, y_pred, labels=class_labels, average=None)
    f1 = f1_score(y, y_pred, labels=class_labels, average=None)
    return precision, recall, f1

# Data preparation
X_train, y_train = preprocess_data(train_df)
X_validation, y_validation = preprocess_data(validation_df)

# Parameter grid
param_grid = {
    'vectorizer__max_df': [0.5, 0.75, 1.0],
    'classifier__alpha': [0.1, 1.0, 10.0],
    'vectorizer__ngram_range': [(1, 1), (1, 2)]
}

results = []

# Iterate through the parameter grid
for params in ParameterGrid(param_grid):
    vectorizer_params = {
        'max_df': params['vectorizer__max_df'],
        'ngram_range': params['vectorizer__ngram_range']
    }
    classifier_params = {
        'alpha': params['classifier__alpha']
    }

    # Create a new pipeline with the specified parameters
    vectorizer = CountVectorizer(**vectorizer_params)
    classifier = MultinomialNB(**classifier_params)
    pipeline = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', classifier)
    ])

    # Fit the pipeline on the training set
    pipeline.fit(X_train, y_train)

    # Evaluate on the validation set
    precision, recall, f1 = evaluate_pipeline(pipeline, X_validation, y_validation)

    results.append({
        'params': params,
        'precision_clickbait': precision[0],
        'recall_clickbait': recall[0],
        'f1_score_clickbait': f1[0],
        'precision_non_clickbait': precision[1],
        'recall_non_clickbait': recall[1],
        'f1_score_non_clickbait': f1[1],
    })

# Print the results (sorted by F1-score for clickbait class)
results_df = pd.DataFrame(results)
sorted_results = results_df.sort_values(by='f1_score_clickbait', ascending=False)
print(sorted_results)


                                               params  precision_clickbait  \
0   {'classifier__alpha': 0.1, 'vectorizer__max_df...             0.835821   
2   {'classifier__alpha': 0.1, 'vectorizer__max_df...             0.835821   
4   {'classifier__alpha': 0.1, 'vectorizer__max_df...             0.835821   
7   {'classifier__alpha': 1.0, 'vectorizer__max_df...             0.833333   
11  {'classifier__alpha': 1.0, 'vectorizer__max_df...             0.833333   
10  {'classifier__alpha': 1.0, 'vectorizer__max_df...             0.833333   
8   {'classifier__alpha': 1.0, 'vectorizer__max_df...             0.833333   
9   {'classifier__alpha': 1.0, 'vectorizer__max_df...             0.833333   
6   {'classifier__alpha': 1.0, 'vectorizer__max_df...             0.833333   
1   {'classifier__alpha': 0.1, 'vectorizer__max_df...             0.808824   
5   {'classifier__alpha': 0.1, 'vectorizer__max_df...             0.808824   
3   {'classifier__alpha': 0.1, 'vectorizer__max_df...           

# Problem 5

Model selection (10pts)

Using these validation-set metrics from the previous problem, choose one model as your selected
model. It is up to you how to choose this model; one approach is to choose the model that shows the
highest F1-score on your training set.

Next apply your selected model to your test set and compute precision, recall and F1. Show results in
your notebook 

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score

# Find the model with the highest F1-score for the 'clickbait' class on the validation set
best_model_idx = sorted_results.index[0]

# Retrieve the best model's parameters
best_params = sorted_results.loc[best_model_idx, 'params']

# Extract the best parameters for vectorizer and classifier
vectorizer_params = best_params['vectorizer__max_df'], best_params['vectorizer__ngram_range']
classifier_params = best_params['classifier__alpha']

# Create the best model pipeline using the selected parameters
best_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_df=vectorizer_params[0], ngram_range=vectorizer_params[1])),
    ('classifier', MultinomialNB(alpha=classifier_params))
])

# Fit the best model on the training set
best_pipeline.fit(X_train, y_train)

# Predict on the test set
X_test = test_df['text']
y_test = test_df['label']
y_test_pred = best_pipeline.predict(X_test)

# Calculate precision, recall, and F1-score for the test set
labels = ['clickbait', 'non-clickbait']
test_precision = precision_score(y_test, y_test_pred, labels=labels, average=None)
test_recall = recall_score(y_test, y_test_pred, labels=labels, average=None)
test_f1 = f1_score(y_test, y_test_pred, labels=labels, average=None)

# Display the results for the test set
for label in labels:
    label_idx = labels.index(label)
    precision = round(test_precision[label_idx] * 100, 4)
    recall = round(test_recall[label_idx] * 100, 4)
    f1 = round(test_f1[label_idx] * 100, 4)
    
    print(f"Test Precision ({label}): {precision}%")
    print(f"Test Recall ({label}): {recall}%")
    print(f"Test F1-Score ({label}): {f1}%")
    print()


Test Precision (clickbait): 83.2215%
Test Recall (clickbait): 86.7133%
Test F1-Score (clickbait): 84.9315%

Test Precision (non-clickbait): 94.2424%
Test Recall (non-clickbait): 92.5595%
Test F1-Score (non-clickbait): 93.3934%



# Problem 6

Key Indicators (10pts)

Using the log-probabilities of the model you selected in the previous problem, select 5 words that are
strong Clickbait indicators. That is, if you needed to filter headlines based on a single word, without a
machine learning model, then these words would be good options. Show this list of keywords in your
notebook.

You can choose how to handle bigrams (e.g., "win<space>big"); you may choose to ignore them and
only select unigram vocabulary words as key indicators. 

In [8]:
import pandas as pd
import re

# Get the trained Naïve Bayes classifier and CountVectorizer from the best pipeline
classifier = best_pipeline.named_steps['classifier']
vectorizer = best_pipeline.named_steps['vectorizer']

# Get the vocabulary from the CountVectorizer
vocab = vectorizer.get_feature_names_out()

# Get the log probabilities of words for the "clickbait" class
log_probabilities = classifier.feature_log_prob_[1]

# Create a DataFrame to store words and their log probabilities
word_probabilities_df = pd.DataFrame({'Word': vocab, 'Log Probability (Clickbait)': log_probabilities})

# Sort the DataFrame by log probabilities in descending order
sorted_word_probabilities = word_probabilities_df.sort_values(by='Log Probability (Clickbait)', ascending=False)

# Get the top 5 words with the highest log probabilities
top_clickbait_words = sorted_word_probabilities.head(5)

# Display the list of top clickbait indicator words
top_clickbait_words_list = top_clickbait_words['Word'].values
print("Top 5 Clickbait Indicator Words:")
print(top_clickbait_words_list)



Top 5 Clickbait Indicator Words:
['to' 'in' 'of' 'for' 'the']


# Problem 7

Regular expressions (10pts)

Your IT department has reached out to you because they heard you can help them find clickbait. They
are interested in your machine learning model, but they need a solution today.

• Write a regular expression that checks if any of the keywords from the previous problem are
found in the text. You should write one regular expression that detects any of your top 5
keywords. Your regular expression should be aware of word boundaries in some way. That is,
the keyword "win" should not be detected in the text "Gas prices up in winter months".

• Using the python re library – apply your function to your test set. (See function re.search). What
is the precision and recall of this classifier? Show your results in your notebook

In [9]:

# Create a regular expression pattern to match any of these keywords with word boundaries
pattern = r'\b(?:' + '|'.join(re.escape(keyword) for keyword in top_clickbait_words_list) + r')\b'

# Initialize counters for true positives, false positives, and false negatives
tp, fp, fn = 0, 0, 0

# Iterate through the test set and apply the regular expression to each text
for text, label in zip(X_test, y_test):
    matched = re.search(pattern, text, flags=re.IGNORECASE)  # Case insensitive match
    if matched and label == 'clickbait':
        tp += 1  # True positive
    elif matched:
        fp += 1  # False positive
    elif label == 'clickbait':
        fn += 1  # False negative

# Calculate precision and recall
precision = tp / (tp + fp)
recall = tp / (tp + fn)

# Display the results
print("Precision:", precision)
print("Recall:", recall)


Precision: 0.2857142857142857
Recall: 0.6573426573426573


# Problem 8


Comparing results (15pts – Answer in Blackboard)

• Compare your rules-based classifier from the previous problem with your machine-learning
solution. Which classifier showed the best model metrics? Why do you think it performed the
best? How did both compare to your trivial baseline (Problem 2)?

• If you had more time to try to improve this clickbait detection solution, what would you
explore? (There is no single correct answer to this question; review your results and come up
with your own ideas)